# Problem 5 Jupyter Helper

# Building a Trie in Python

Before we start let us reiterate the key components of a Trie or Prefix Tree. A trie is a tree-like data structure that stores a dynamic set of strings. Tries are commonly used to facilitate operations like predictive text or autocomplete features on mobile phones or web search.

Before we move into the autocomplete function we need to create a working trie for storing strings.  We will create two classes:
* A `Trie` class that contains the root node (empty string)
* A `TrieNode` class that exposes the general functionality of the Trie, like inserting a word or finding the node which represents a prefix.

Give it a try by implementing the `TrieNode` and `Trie` classes below!

In [57]:
## Represents a single node in the Trie
class TrieNode:
    def __init__(self):
        ## Initialize this node in the Trie
        self.is_word = False
        self.children = {}
    
    def insert_char(self, char):
        ## Add a child node in this Trie
        assert type(char) == str, f'''Inserted character must be string!'''
        assert len(char) <= 1, f'''
        The length of the character must be less than or equal to 1!
        '''
        self.children[char] = TrieNode()

In [58]:
## The Trie itself containing the root node and insert/find functions
class Trie:
    def __init__(self):
        ## Initialize this Trie (add a root node)
        self.root = TrieNode()

    def insert_word(self, word):
        ## Add a word to the Trie
        current_node = self.root
        
        for char in word:
            if char not in current_node.children:
                current_node.insert_char(char)
            current_node = current_node.children[char]
        
        current_node.is_word = True

    def find(self, prefix):
        ## Find the Trie node that represents this prefix
        current_node = self.root
        
        for char in prefix:
            if char not in current_node.children:
                return None
            current_node = current_node.children[char]
        
        return current_node

In [59]:
MyTrie = Trie()
wordList = [
    "ant", "anthology", "antagonist", "antonym", 
    "fun", "function", "factory", 
    "trie", "trigger", "trigonometry", "tripod"
]
for word in wordList:
    MyTrie.insert_word(word)

# Finding Suffixes

Now that we have a functioning Trie, we need to add the ability to list suffixes to implement our autocomplete feature.  To do that, we need to implement a new function on the `TrieNode` object that will return all complete word suffixes that exist below it in the trie.  For example, if our Trie contains the words `["fun", "function", "factory"]` and we ask for suffixes from the `f` node, we would expect to receive `["un", "unction", "actory"]` back from `node.suffixes()`.

Using the code you wrote for the `TrieNode` above, try to add the suffixes function below. (Hint: recurse down the trie, collecting suffixes as you go.)

In [60]:
class TrieNode:
    def __init__(self):
        ## Initialize this node in the Trie
        self.is_word = False
        self.children = {}
    
    def insert_char(self, char):
        ## Add a child node in this Trie
        assert type(char) == str, f'''Inserted character must be string!'''
        assert len(char) <= 1, f'''
        The length of the character must be less than or equal to 1!
        '''
        self.children[char] = TrieNode()
    
    def has_children(self):
        return len(self.children) > 0
    
    def empty_suffix(self, the_string):
        return len(the_string) < 1
        
    def suffixes(self, suffix = '', current_node = None):
        ## Recursive function that collects the suffix for 
        ## all complete words below this point
        
        # This conditional determines whether there is a suffix.
        if self.empty_suffix(suffix):
            # This initiates the list_of_suffixes.
            list_of_suffixes = []
            
            # If this node has no children, an empty list_of_suffixes is
            # returned.
            if not self.has_children():
                return list_of_suffixes
            
            # If this node has children, then a for loop is activated to
            # search for suffixes.
            char_dict = self.children
            for char in char_dict:
                current_suffix = suffix + char
                the_current_node = self.children[char]
                if current_node.is_word:
                    list_of_suffixes.append(current_suffix)
                output = self.suffixes(current_suffix, the_current_node)
                if output:
                    for i in output:
                        list_of_suffixes.append(output)
            return list_of_suffixes
        
        if not current_node.has_children():
            if current_node.is_word:
                return [suffix]
            return
        
        char_dict = current_node.children
        more_outputs = []
        for char in char_dict:
            current_suffix = suffix + char
            the_current_node = current_node.children[char]
            if current_node.is_word:
                more_outputs.append(current_suffix)
            output = 
            self.suffixes(current_suffix, the_current_node)

In [55]:
no_children = TrieNode()

In [56]:
no_children.has_children()

False

# Testing it all out

Run the following code to add some words to your trie and then use the interactive search box to see what your code returns.

In [62]:
MyTrie = Trie()
wordList = [
    "ant", "anthology", "antagonist", "antonym", 
    "fun", "function", "factory", 
    "trie", "trigger", "trigonometry", "tripod"
]
for word in wordList:
    MyTrie.insert_word(word)

## DO NOT RUN CELLS BELOW FOR NOW!

In [64]:
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import interact
def f(prefix):
    if prefix != '':
        prefixNode = MyTrie.find(prefix)
        if prefixNode:
            print('\n'.join(prefixNode.suffixes()))
        else:
            print(prefix + " not found")
    else:
        print('')
interact(f,prefix='');

interactive(children=(Text(value='', description='prefix'), Output()), _dom_classes=('widget-interact',))

In [8]:
# REFERENCES
# 1. Udacity Data Structures & Algorithms Nanodegree; 3) Basic Algorithms; 
#        1) Basic Algorithms; 6) Tries
# 2. Udacity Data Structures & Algorithms Nanodegree; 3) Basic Algorithms; 
#        4) Problems Vs Algorithms; 6) Problem 5: Autocomplete with Tries